# European Develompent Indicators - Iceland

## Research Questions

1. How has Iceland's GDP growth correlated with inflation rates over the past 20 years?

2. What is the relationship between educational attainment and unemployment in Iceland?

3. What is the gender wage gap trend in Iceland, and how does it compare with female employment rates?

## Datasets

* [namq_10_gdp](https://doi.org/10.2908/NAMQ_10_GDP) - Eurostat: Quarterly GDP

* [nama_10_co3_p3](https://doi.org/10.2908/NAMA_10_CO3_P3) - Eurostat: Annual household disposable income

* [tec00001](https://doi.org/10.2908/TEC00001) - Eurostat: Annual GDP at market prices

* [prc_hicp_midx](https://doi.org/10.2908/PRC_HICP_MIDX) - Eurostat: Monthly Harmonized CPI

* [prc_hicp_manr](https://doi.org/10.2908/PRC_HICP_MANR) - Eurostat: Monthly CPI inflation rate

* [prc_hicp_inw](https://doi.org/10.2908/PRC_HICP_INW) - Eurostat: Annual CPI inflation rate by income group

* [lfsa_urgaed](https://doi.org/10.2908/LFSA_URGAED) - Eurostat: Annual unemployment rate by education level

* [lfsa_egaed](https://doi.org/10.2908/LFSA_EGAED) - Eurostat: Annual employment rate by education level

* [lfsa_egised](https://doi.org/10.2908/LFSA_EGISED) - Eurostat: Annual employed persons by occupation and educational attainment level

* [lfsa_egan](https://doi.org/10.2908/LFSA_EGAN) - Eurostat: Annual employment

* [lfsi_emp_q](https://doi.org/10.2908/LFSI_EMP_Q) - Eurostat: Quarterly employment and activity by sex and age

* [lfsi_long_q](https://doi.org/10.2908/LFSI_LONG_Q) - Eurostat: Quarterly labour market transitions

* [lfsq_ergan](https://doi.org/10.2908/LFSQ_ERGAN) - Eurostat: Quarterly employment rates by citizenship

* [une_rt_m](https://doi.org/10.2908/UNE_RT_M) - Eurostat: Monthly total unemployment rate

* [edat_lfse_03](https://doi.org/10.2908/EDAT_LFSE_03) - Eurostat: Annual population by educational attainment level

* [earn_nt_net](https://doi.org/10.2908/EARN_NT_NET) - Eurostat: Annual net earnings

* [earn_gr_gpgr2](https://doi.org/10.2908/EARN_GR_GPGR2) - Eurostat: Annual gender pay gap in unadjusted form


## Imports

In [2]:
import os
import eurostat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from src.frame_class import Frame, Frequency, Category

## Parameters

In [ ]:
COUNTRY = {
    "name": "Iceland",
    "code": "IS"
}
DATAFRAMES = [
    Frame(
        name="Quarterly GDP",
        file_name="quarterly_gdp.csv",
        description="Gross domestic product (GDP) and main components (output, expenditure and income)",
        eurostat_code="namq_10_gdp",
        frequency=Frequency.QUARTERLY,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual Household Disposable Income",
        file_name="annual_household_disposable_income.csv",
        description="Household final consumption expenditure by purpose (COICOP 1999)",
        eurostat_code="nama_10_co3_p3",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual GDP at Market Prices",
        file_name="annual_gdp_at_market_prices.csv",
        description="Gross domestic product at market prices",
        eurostat_code="tec00001",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Monthly Harmonized CPI",
        file_name="monthly_harmonized_cpi.csv",
        description="HICP - monthly data (index)",
        eurostat_code="prc_hicp_midx",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Monthly CPI Inflation Rate",
        file_name="monthly_cpi_inflation_rate.csv",
        description="HICP - monthly data (annual rate of change)",
        eurostat_code="prc_hicp_manr",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Monthly CPI Inflation Rate by Income Group",
        file_name="monthly_cpi_inflation_rate_by_income_group.csv",
        description="HICP - item weights",
        eurostat_code="prc_hicp_inw",
        frequency=Frequency.ANNUAL,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Quarterly Unemployment Rate by Education Level",
        file_name="quarterly_unemployment_rate_by_education_level.csv",
        description="Unemployment rates by educational attainment level",
        eurostat_code="lfsa_urgaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Quarterly Employment Rate by Education Level",
        file_name="quarterly_employment_rate_by_education_level.csv",
        description="Employed persons by educational attainment level",
        eurostat_code="lfsa_egaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Quarterly Employed Persons by Occupation and Educational Attainment Level",
        file_name="quarterly_employed_persons_by_occupation_and_educational_attainment_level.csv",
        description="Employed persons by occupation and educational attainment level",
        eurostat_code="lfsa_egised",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Yearly Employment",
        file_name="yearly_employment.csv",
        description="Employed persons by citizenship",
        eurostat_code="lfsa_egan",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment and activity by sex and age",
        file_name="quarterly_employment_and_activity_by_sex_and_age.csv",
        description="Employment and activity by sex and age - quarterly data",
        eurostat_code="lfsi_emp_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly labour market transitions",
        file_name="quarterly_labour_market_transitions.csv",
        description="Labour market transitions - quarterly data",
        eurostat_code="lfsi_long_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment rates by citizenship",
        file_name="quarterly_employment_rates_by_citizenship.csv",
        description="Employment rates by citizenship - quarterly data",
        eurostat_code="lfsq_ergan",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Monthly total unemployment rate",
        file_name="monthly_total_unemployment_rate.csv",
        description="Unemployment by sex and age - monthly data",
        eurostat_code="une_rt_m",
        frequency=Frequency.MONTHLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Annual population by educational attainment level",
        file_name="annual_population_by_educational_attainment_level.csv",
        description="Population in private households by educational attainment level - main indicators",
        eurostat_code="edat_lfse_03",
        frequency=Frequency.ANNUAL,
        categories=[Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual net earnings",
        file_name="annual_net_earnings.csv",
        description="Annual net earnings",
        eurostat_code="earn_nt_net",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    ),
    Frame(
        name="Annual gender pay gap in unadjusted form",
        file_name="annual_gender_pay_gap_in_unadjusted_form.csv",
        description="Gender pay gap in unadjusted form by NACE Rev. 2 activity - structure of earnings survey methodology",
        eurostat_code="earn_gr_gpgr2",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    )
]

DATA_PATH = "data/"

print(f"Frame count: {len(DATAFRAMES)}")

Frame count: 17


In [4]:
def load_dataframe(frame: Frame) -> pd.DataFrame | None:
    df: pd.DataFrame | None = None

    if os.path.exists(os.path.join("original_data", frame.file_name)):
        df = pd.read_csv(os.path.join("original_data", frame.file_name))
        print(f"Loaded {frame.name} dataset from local CSV.")

    else:
        try:
            df = eurostat.get_data_df(frame.eurostat_code, flags=False)
            os.makedirs("original_data", exist_ok=True)
            df.to_csv(os.path.join("original_data", frame.file_name), index=False)

            print(f"Downloaded and saved {frame.name} dataset.")

        except Exception as e:
            print(f"Failed to download {frame.name}: {e}")

    df = df[df["geo\\TIME_PERIOD"] == COUNTRY["code"]]

    return df


In [5]:
for frame in DATAFRAMES:
    df: pd.DataFrame | None = None

    if os.path.exists(os.path.join("original_data", frame.file_name)):
        df = pd.read_csv(os.path.join("original_data", frame.file_name))
        print(f"Loaded {frame.name} dataset from local CSV.")

    else:
        try:
            df = eurostat.get_data_df(frame.eurostat_code, flags=False)
            os.makedirs("original_data", exist_ok=True)
            df.to_csv(os.path.join("original_data", frame.file_name), index=False)

            print(f"Downloaded and saved {frame.name} dataset.")

        except Exception as e:
            print(f"Failed to download {frame.name}: {e}")

    df = df[df["geo\\TIME_PERIOD"] == COUNTRY["code"]]

    frame.dataframe = df

    os.makedirs(DATA_PATH, exist_ok=True)
    frame.dataframe.to_csv(os.path.join(DATA_PATH, frame.file_name), index=False)


Loaded Quarterly GDP dataset from local CSV.
Loaded Annual Household Disposable Income dataset from local CSV.
Loaded Annual GDP at Market Prices dataset from local CSV.
Loaded Monthly Harmonized CPI dataset from local CSV.
Loaded Monthly CPI Inflation Rate dataset from local CSV.
Loaded Monthly CPI Inflation Rate by Income Group dataset from local CSV.
Loaded Quarterly Unemployment Rate by Education Level dataset from local CSV.
Loaded Quarterly Employment Rate by Education Level dataset from local CSV.
Loaded Quarterly Employed Persons by Occupation and Educational Attainment Level dataset from local CSV.
Loaded Yearly Employment dataset from local CSV.
Loaded Quarterly employment and activity by sex and age dataset from local CSV.
Loaded Quarterly labour market transitions dataset from local CSV.
Loaded Quarterly employment rates by citizenship dataset from local CSV.
Loaded Monthly total unemployment rate dataset from local CSV.
Loaded Annual population by educational attainment le

In [ ]:
for frame in DATAFRAMES.filter(lambda f: f.frequency == Frequency.QUARTERLY):
    # get the column names after the "geo\TIME_PERIOD" column
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo\\TIME_PERIOD")
    data_columns = column_names[time_period_index + 1 :]

    